In [1]:
import json
import boto3
import pandas as pd
from getpass import getpass
from configparser import ConfigParser

# Loading Params from a config file

In [2]:
config = ConfigParser()
config.read_file(open('dwh.cfg'))

# CREDENTIALS
print("Enter your ACCESS KEY: ")
KEY                = getpass()
print("Enter your SECRET KEY: ")
SECRET             = getpass()

# REDSHIFT CLUSTER
CLUSTER_IDENTIFIER = config["CLUSTER"]["CLUSTER_IDENTIFIER"]
CLUSTER_TYPE       = config["CLUSTER"]["CLUSTER_TYPE"]
NUM_NODES          = config["CLUSTER"]["NUM_NODES"]
NODE_TYPE          = config["CLUSTER"]["NODE_TYPE"]

# DATABASE INFOS
DB_NAME            = config["DATABASE"]["DB_NAME"]
DB_USER            = config["DATABASE"]["DB_USER"]
DB_PASSWORD        = config["DATABASE"]["DB_PASSWORD"]
DB_PORT            = config["DATABASE"]["DB_PORT"]

# IAM ROLE
IAM_ROLE_NAME      = config["IAM_ROLE"]["NAME"]

# REGION
REGION             = config["REGION"]["NAME"]

pd.DataFrame({
    "Param": [
        "CLUSTER_IDENTIFIER",
        "CLUSTER_TYPE",
        "NUM_NODES",
        "NODE_TYPE",
        "DB_NAME",
        "DB_USER",
        "DB_PASSWORD",
        "DB_PORT",
        "IAM_ROLE_NAME"
    ],
    "Value": [
        CLUSTER_IDENTIFIER,
        CLUSTER_TYPE, 
        NUM_NODES, 
        NODE_TYPE,  
        DB_NAME, 
        DB_USER, 
        DB_PASSWORD, 
        DB_PORT, 
        IAM_ROLE_NAME
    ]
})

Enter your ACCESS KEY: 


 ····················


Enter your SECRET KEY: 


 ········································


,Param,Value
0,CLUSTER_IDENTIFIER,redshiftCluster
1,CLUSTER_TYPE,multi-node
2,NUM_NODES,4
3,NODE_TYPE,dc2.large
4,DB_NAME,redshiftdb
5,DB_USER,redshiftuser
6,DB_PASSWORD,Passw0rd
7,DB_PORT,5439
8,IAM_ROLE_NAME,redshiftRole


# Create connections for IAM, EC2, S3 and Redshift

In [3]:
ec2 = boto3.resource('ec2',
                     region_name=REGION,
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                    region_name=REGION,
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',
                   region_name=REGION,
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

redshift = boto3.client('redshift',
                        region_name=REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                       )

# Create IAM Role

In [4]:
try:
    # Creating a new IAM Role
    roleResponse = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description="Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps({
            'Statement': [{
                'Action': 'sts:AssumeRole',
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'}
            }],
            'Version': '2012-10-17'
        })
    )
    
    if roleResponse['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'{IAM_ROLE_NAME} has been created successfully')
        
        # Get the IAM role ARN
        IAM_ROLE_ARN = roleResponse['Role']['Arn']
        print('\nIAM ROLE ARN: ', IAM_ROLE_ARN)
    
except Exception as e:
    print(e)

try:
    # Attaching policy to IAM ROLE
    policyResponse = iam.attach_role_policy(
        RoleName=IAM_ROLE_NAME,
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    )
    
    if policyResponse['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'\nPolicy has been successfully attached to {IAM_ROLE_NAME}')

except Exception as e:
    print(e)

redshiftRole has been created successfully

IAM ROLE ARN:  arn:aws:iam::564727721921:role/redshiftRole

Policy has been successfully attached to redshiftRole


# Create Redshift Cluster

In [5]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[IAM_ROLE_ARN]  
    )
    
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'{CLUSTER_IDENTIFIER} has been created successfully')
        
except Exception as e:
    print(e)

redshiftCluster has been created successfully


In [6]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", 
                  "DBName", "Endpoint", "IamRoles", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

# Run cell below until the cluster is available then run the next cell
This may take around 3 minutes

In [7]:
cluster_status = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]['ClusterStatus']
cluster_status

'available'

# Check cluster details

In [8]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
ENDPOINT = myClusterProps['Endpoint']['Address']
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,redshiftcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,redshiftuser
4,DBName,redshiftdb
5,Endpoint,"{'Address': 'redshiftcluster.crjn8bqxq38c.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-06be3371c6557ac71
7,NumberOfNodes,4
8,IamRoles,"[{'IamRoleArn': 'arn:aws:iam::564727721921:role/redshiftRole', 'ApplyStatus': 'in-sync'}]"


# Updating dwh.cfg file

In [9]:
# Get the IAM_ROLE section
iamRole = config['IAM_ROLE']
# Get database section
database = config['DATABASE']

# Update the ARN
iamRole['ARN'] = IAM_ROLE_ARN
# Update the HOST
database['HOST'] = ENDPOINT

# Write changes back to file
with open('dwh.cfg', 'w') as editfile:
    config.write(editfile)

# Open an incoming  TCP port to access the cluster endpoint

In [10]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

# Clean up your resources

## Delete redshift cluster
Run cell below until the cluster is deleted. This may take few minutes.

In [14]:
try:
    status = redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)['Cluster']['ClusterStatus']
    print(status)
except:
    print("The cluster was successfully deleted")

The cluster was successfully deleted


## Delete IAM Role

In [15]:
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)['ResponseMetadata']['HTTPStatusCode']

200